# Position Refinement

**Learning how to relax the exact location of the scan positions to correct for instabilities and drift.**

---

In [ ]:
import ptypy, os
import ptypy.utils as u

# This will import the HDF5Loader class
ptypy.load_ptyscan_module("hdf5_loader")

# This will import the GPU engines
ptypy.load_gpu_engines("cuda")  

# Root directory of tutorial data
tutorial_data_home = "/dls/science/groups/imaging/ptypy_tutorials/"

# Path to HDF5 file with raw data
dataset = "dls_i14_test_structure/i14-112767.nxs"

# Path to a previous reconstruction
init_probe = "dls_i14_test_structure/scan_103937.ptyr"

# Path to HDF5 file with mask
mask = "dls_i14_test_structure/mask-I14-merlin-112767.h5"

# Absolute paths to HDF5 files with raw data, initial probe and mask
path_to_data = os.path.join(tutorial_data_home, dataset)
path_to_probe = os.path.join(tutorial_data_home, init_probe)
path_to_mask = os.path.join(tutorial_data_home, mask)

# Create parameter tree
p = u.Param()

# Set verbose level to info
p.verbose_level = "interactive"

# Scan label
p.run = "dls_i14_test_structure"

# Data loading and processing should 
# happen in chunks of this size
p.frames_per_block = 10000

# Set io settings (no files saved)
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.autoplot = u.Param(active=False)
p.io.interaction = u.Param(active=False)

# Define the scan model
p.scans = u.Param()
p.scans.scan_00 = u.Param()
p.scans.scan_00.name = 'BlockFull'

# Initial illumination (based on previous reconstruction)
p.scans.scan_00.illumination = u.Param()
p.scans.scan_00.illumination.model = "recon"
p.scans.scan_00.illumination.recon = u.Param()
p.scans.scan_00.illumination.recon.rfile = path_to_probe
p.scans.scan_00.illumination.photons = None
p.scans.scan_00.illumination.aperture = u.Param()
p.scans.scan_00.illumination.aperture.form = None
p.scans.scan_00.illumination.diversity = u.Param()
p.scans.scan_00.illumination.diversity.power = 0.1
p.scans.scan_00.illumination.diversity.noise = [0.5,0.2]

# Initial object
p.scans.scan_00.sample = u.Param()
p.scans.scan_00.sample.model = None
p.scans.scan_00.sample.diversity = None
p.scans.scan_00.sample.process = None

# Coherence parameters (modes)
p.scans.scan_00.coherence = u.Param()
p.scans.scan_00.coherence.num_probe_modes = 1
p.scans.scan_00.coherence.num_object_modes = 1

# Data loader
p.scans.scan_00.data = u.Param()
p.scans.scan_00.data.name = 'Hdf5LoaderFast'
p.scans.scan_00.data.orientation = 0

p.scans.scan_00.data.intensities = u.Param()
p.scans.scan_00.data.intensities.file = path_to_data
p.scans.scan_00.data.intensities.key = "entry/merlin_addetector/data"

p.scans.scan_00.data.positions = u.Param()
p.scans.scan_00.data.positions.file = path_to_data
p.scans.scan_00.data.positions.slow_key = "entry/xsp3_addetector_sum/SampleY_value_set"
p.scans.scan_00.data.positions.slow_multiplier = 1e-3
p.scans.scan_00.data.positions.fast_key = "entry/xsp3_addetector_sum/SampleX_value_set"
p.scans.scan_00.data.positions.fast_multiplier = 1e-3
p.scans.scan_00.data.positions.bounding_box = u.Param()
p.scans.scan_00.data.positions.bounding_box.slow_axis_bounds = [200,400]
p.scans.scan_00.data.positions.bounding_box.fast_axis_bounds = [250,500]

p.scans.scan_00.data.recorded_energy = u.Param()
p.scans.scan_00.data.recorded_energy.key = "/entry/instrument/dcm_enrg/value"
p.scans.scan_00.data.recorded_energy.file = path_to_data
p.scans.scan_00.data.recorded_energy.multiplier = 1

p.scans.scan_00.data.recorded_distance = u.Param()
p.scans.scan_00.data.recorded_distance.key = "/entry/instrument/detectors/excalibur_z"
p.scans.scan_00.data.recorded_distance.file = path_to_data
p.scans.scan_00.data.recorded_distance.multiplier = 0.001

p.scans.scan_00.data.mask = u.Param()
p.scans.scan_00.data.mask.file = path_to_mask
p.scans.scan_00.data.mask.key = "data/data"

p.scans.scan_00.data.psize = 55e-6
p.scans.scan_00.data.auto_center = False
p.scans.scan_00.data.center = (115, 333)
p.scans.scan_00.data.shape = (128,128)
p.scans.scan_00.data.save = None

# Reconstruct using GPU-accelerated DM
p.engines = u.Param()
p.engines.engine = u.Param()
p.engines.engine.name = "DM_pycuda"
p.engines.engine.numiter = 300
p.engines.engine.numiter_contiguous = 10
p.engines.engine.alpha = 0.99
p.engines.engine.probe_support = None
p.engines.engine.probe_fourier_support = None
p.engines.engine.overlap_converge_factor = 0.001
p.engines.engine.probe_update_start = 0
p.engines.engine.update_object_first = True
p.engines.engine.obj_smooth_std = 20
p.engines.engine.probe_inertia = 0.001
p.engines.engine.object_inertia = 0.001
p.engines.engine.fourier_power_bound = 0.25
p.engines.engine.record_local_error = False
#p.engines.engine.fft_lib = "cuda"

# Position refinement
p.engines.engine.position_refinement = u.Param()
p.engines.engine.position_refinement.start = 100
p.engines.engine.position_refinement.stop = 200
p.engines.engine.position_refinement.interval = 10
p.engines.engine.position_refinement.nshifts = 8
p.engines.engine.position_refinement.amplitude = 50.0e-9
p.engines.engine.position_refinement.max_shift = 100.0e-9
p.engines.engine.position_refinement.record = False

# Run reconstruction
P = ptypy.core.Ptycho(p,level=5)

# Display the results
u.plot_recon_from_ptycho(P)